DEEP FOOL ATTACK

In [40]:

from __future__ import absolute_import, division, print_function, unicode_literals

import logging

import tensorflow as tf

tf.compat.v1.disable_eager_execution()

from keras.models import Sequential
from keras.layers import Dense, Flatten, Conv2D, MaxPooling2D, Activation, Dropout
import numpy as np

from art.attacks.evasion import DeepFool
from art.estimators.classification import KerasClassifier
from art.utils import load_dataset

In [41]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Input, Conv2D, BatchNormalization, MaxPooling2D, Flatten, Dense
from tensorflow.keras.losses import sparse_categorical_crossentropy
from tensorflow.keras.metrics import sparse_categorical_accuracy
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.regularizers import l2
from tensorpack.dataflow import dataset, BatchData
from tensorpack.train import TrainConfig
from tensorpack.tfutils.varreplace import remap_variables
from tensorpack.tfutils.scope_utils import auto_reuse_variable_scope
from tensorpack.callbacks import ModelSaver, MaxSaver, InferenceRunner
from tensorflow.keras.datasets import mnist

In [42]:
# Configure a logger to capture ART outputs; these are printed in console and the level of detail is set to INFO
logger = logging.getLogger()
logger.setLevel(logging.INFO)
handler = logging.StreamHandler()
formatter = logging.Formatter("[%(levelname)s] %(message)s")
handler.setFormatter(formatter)
logger.addHandler(handler)

In [43]:
from sklearn.model_selection import train_test_split

# Read CIFAR-10 dataset
(x_train, y_train), (x_test, y_test), min_, max_ = load_dataset("mnist")

# Shuffle and split the dataset
x_train, x_test, y_train, y_test = train_test_split(
    x_train, y_train, test_size=0.2, random_state=42
)

im_shape = x_train[0].shape

In [20]:
# Preprocess the data
x_train = x_train / 255.0
x_test = x_test / 255.0
# Define constants
IMAGE_SIZE = 28 # Example value, replace with your actual image size

model = Sequential()
model.add(Input(shape=(IMAGE_SIZE, IMAGE_SIZE, 1), name='input'))  # Input layer

model.add(Conv2D(32, (3, 3), use_bias=False, activation='relu', kernel_regularizer=l2(1e-5), name='conv1'))
model.add(BatchNormalization(name='bn1'))
model.add(MaxPooling2D(pool_size=(2, 2), name='maxpool1'))  # Reduced max-pooling

model.add(Conv2D(64, (3, 3), use_bias=False, activation='relu', kernel_regularizer=l2(1e-5), name='conv2'))
model.add(BatchNormalization(name='bn2'))
model.add(MaxPooling2D(pool_size=(2, 2), name='maxpool2'))  # Reduced max-pooling

model.add(Conv2D(128, (3, 3), use_bias=False, activation='relu', kernel_regularizer=l2(1e-5), name='conv3'))
model.add(BatchNormalization(name='bn3'))

model.add(Conv2D(256, (3, 3), use_bias=False, activation='relu', kernel_regularizer=l2(1e-5), name='conv4'))
model.add(BatchNormalization(name='bn4'))

model.add(Flatten(name='flatten'))
model.add(Dense(20, use_bias=False, activation='relu', kernel_regularizer=l2(1e-5), name='fc1'))
model.add(Dense(10, use_bias=False, activation='softmax', name='output'))

# Print the model summary
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1 (Conv2D)              (None, 26, 26, 32)        288       
                                                                 
 bn1 (BatchNormalization)    (None, 26, 26, 32)        128       
                                                                 
 maxpool1 (MaxPooling2D)     (None, 13, 13, 32)        0         
                                                                 
 conv2 (Conv2D)              (None, 11, 11, 64)        18432     
                                                                 
 bn2 (BatchNormalization)    (None, 11, 11, 64)        256       
                                                                 
 maxpool2 (MaxPooling2D)     (None, 5, 5, 64)          0         
                                                                 
 conv3 (Conv2D)              (None, 3, 3, 128)        

In [21]:

# Compile the model
model.compile(
    optimizer=SGD(learning_rate=0.01, momentum=0.9),
    loss='sparse_categorical_crossentropy',
    metrics=['sparse_categorical_accuracy']
)

# Create classifier wrapper
classifier = KerasClassifier(model=model, clip_values=(min_, max_))
classifier.fit(x_train, y_train, nb_epochs=10, batch_size=128)



[INFO] Inferred 12 hidden layers on Keras classifier.
[INFO] Inferred 12 hidden layers on Keras classifier.


Train on 48000 samples
Epoch 1/10
48000/48000 [==============================] - 5s 105us/sample - loss: 0.1470 - sparse_categorical_accuracy: 0.9572
Epoch 2/10
48000/48000 [==============================] - 5s 101us/sample - loss: 0.0464 - sparse_categorical_accuracy: 0.9877
Epoch 3/10
48000/48000 [==============================] - 5s 99us/sample - loss: 0.0314 - sparse_categorical_accuracy: 0.9921
Epoch 4/10
48000/48000 [==============================] - 5s 99us/sample - loss: 0.0215 - sparse_categorical_accuracy: 0.9945
Epoch 5/10
48000/48000 [==============================] - 5s 99us/sample - loss: 0.0168 - sparse_categorical_accuracy: 0.9960
Epoch 6/10
48000/48000 [==============================] - 5s 99us/sample - loss: 0.0129 - sparse_categorical_accuracy: 0.9976
Epoch 7/10
48000/48000 [==============================] - 5s 99us/sample - loss: 0.0096 - sparse_categorical_accuracy: 0.9987
Epoch 8/10
48000/48000 [==============================] - 5s 101us/sample - loss: 0.0077 - sp

In [22]:
# Evaluate the classifier on the adversarial samples
preds = np.argmax(classifier.predict(x_test), axis=1)
acc = np.sum(preds == np.argmax(y_test, axis=1)) / y_test.shape[0]
logger.info("Classifier test accuracy")
logger.info("Accuracy on test samples: %.2f%%", (acc * 100))

[INFO] Classifier test accuracy
[INFO] Classifier test accuracy
[INFO] Accuracy on test samples: 99.01%
[INFO] Accuracy on test samples: 99.01%


In [7]:
# Craft adversarial samples with DeepFool
logger.info("Create DeepFool attack")
adv_crafter = DeepFool(classifier)
logger.info("Craft attack on training examples")
x_train_adv = adv_crafter.generate(x_train)

[INFO] Create DeepFool attack
[INFO] Craft attack on training examples
/home/user/anaconda3/envs/attack/lib/python3.10/site-packages/keras/engine/training_v1.py:2079: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  updates=self.state_updates,
[WARNING] It seems that the attacked model is predicting probabilities. DeepFool expects logits as model output to achieve its full attack strength.
DeepFool: 100%|██████████| 48000/48000 [4:09:26<00:00,  3.21it/s]   


In [8]:

logger.info("Craft attack test examples")
x_test_adv = adv_crafter.generate(x_test)

[INFO] Craft attack test examples
[WARNING] It seems that the attacked model is predicting probabilities. DeepFool expects logits as model output to achieve its full attack strength.
DeepFool: 100%|██████████| 12000/12000 [1:01:28<00:00,  3.25it/s]


In [9]:

# Evaluate the classifier on the adversarial samples
preds = np.argmax(classifier.predict(x_test_adv), axis=1)
acc = np.sum(preds == np.argmax(y_test, axis=1)) / y_test.shape[0]
logger.info("Classifier before adversarial training")
logger.info("Accuracy on adversarial samples: %.2f%%", (acc * 100))

[INFO] Classifier before adversarial training
[INFO] Accuracy on adversarial samples: 10.61%


DEFENSE USING ADVERSIAL TRAINING

In [11]:
# Data augmentation: expand the training set with the adversarial samples
x_train = np.append(x_train, x_train_adv, axis=0)
y_train = np.append(y_train, y_train, axis=0)

In [12]:
# Retrain the CNN on the extended dataset
model.compile(loss="sparse_categorical_crossentropy", optimizer="adam", metrics=["accuracy"])
classifier.fit(x_train, y_train, nb_epochs=10, batch_size=128)
classifier.save("Adversial_train_model.h5")

Train on 96000 samples
Epoch 1/10
96000/96000 [==============================] - 9s 97us/sample - loss: 0.8014 - accuracy: 0.7527
Epoch 2/10
96000/96000 [==============================] - 9s 95us/sample - loss: 0.3807 - accuracy: 0.8853
Epoch 3/10
96000/96000 [==============================] - 9s 96us/sample - loss: 0.2872 - accuracy: 0.9131
Epoch 4/10
96000/96000 [==============================] - 9s 97us/sample - loss: 0.2448 - accuracy: 0.9267
Epoch 5/10
96000/96000 [==============================] - 9s 97us/sample - loss: 0.2037 - accuracy: 0.9374
Epoch 6/10
96000/96000 [==============================] - 9s 97us/sample - loss: 0.2021 - accuracy: 0.9394
Epoch 7/10
96000/96000 [==============================] - 9s 97us/sample - loss: 0.1611 - accuracy: 0.9513
Epoch 8/10
96000/96000 [==============================] - 9s 96us/sample - loss: 0.1653 - accuracy: 0.9508
Epoch 9/10
96000/96000 [==============================] - 9s 97us/sample - loss: 0.1540 - accuracy: 0.9556
Epoch 10/10
96

[INFO] Model saved in path: /home/user/.art/data/Adversial_train_model.h5.


In [14]:
classifier.save("/home/user/Desktop/vamsi/White_box_Attack/MNIST_FINAL/Adversial_train_model.h5")

[INFO] Model saved in path: /home/user/Desktop/vamsi/White_box_Attack/MNIST_FINAL/Adversial_train_model.h5.


In [15]:
#Evaluate the adversarially trained classifier on the test set
preds = np.argmax(classifier.predict(x_test_adv), axis=1)
acc = np.sum(preds == np.argmax(y_test, axis=1)) / y_test.shape[0]
logger.info("Classifier with adversarial training")
logger.info("Accuracy on adversarial samples: %.2f%%", (acc * 100))

/home/user/anaconda3/envs/attack/lib/python3.10/site-packages/keras/engine/training_v1.py:2079: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  updates=self.state_updates,
[INFO] Classifier with adversarial training
[INFO] Accuracy on adversarial samples: 89.33%


DEFENSIVE DISTILLATION

In [49]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
import keras
from keras.datasets import mnist
from keras.models import Sequential
import tensorflow.keras.optimizers as optimizers
from keras.layers import Dense, Dropout, Flatten, Lambda
from keras.layers import Conv2D, MaxPooling2D
from keras import backend as K
import tensorflow as tf
from scipy.special import softmax
import logging
tf.get_logger().setLevel(logging.ERROR)

config = tf.compat.v1.ConfigProto()

config.gpu_options.allow_growth=True
sess = tf.compat.v1.Session(config=config)

from keras.models import load_model
import matplotlib.pyplot as plt
import matplotlib.style as style
style.use('fivethirtyeight')
import pickle as pkl

batch_size = 128
num_classes = 10
epochs = 10
img_rows, img_cols = 28, 28
# temperature = 20

colors = [[0,0,0], [230/255,159/255,0], [86/255,180/255,233/255], [0,158/255,115/255],
          [213/255,94/255,0], [0,114/255,178/255]]



import numpy as np
from sklearn.model_selection import train_test_split
from keras.datasets import mnist

# Load the MNIST dataset
(x_train, y_train), (x_test, y_test) = mnist.load_data()

x_train = x_train.astype('float32')
x_test = x_test.astype('float32')

# Normalize pixel values to be between 0 and 1
x_train = x_train / 255.0
x_test = x_test / 255.0

# Reshape the data to match your model's input shape
x_train = x_train.reshape(x_train.shape[0], 28, 28, 1)
x_test = x_test.reshape(x_test.shape[0], 28, 28, 1)

# Split the dataset into training and testing sets (80% training, 20% testing)
x_train, x_test, y_train, y_test = train_test_split(x_train, y_train, test_size=0.2, random_state=42)



y_train = tf.keras.utils.to_categorical(y_train, num_classes)
y_test = tf.keras.utils.to_categorical(y_test, num_classes)


if K.image_data_format() == 'channels_first':
    x_train = x_train.reshape(x_train.shape[0], 1, img_rows, img_cols)
    x_test = x_test.reshape(x_test.shape[0], 1, img_rows, img_cols)
    input_shape = (1, img_rows, img_cols)
else:
    x_train = x_train.reshape(x_train.shape[0], img_rows, img_cols, 1)
    x_test = x_test.reshape(x_test.shape[0], img_rows, img_cols, 1)
    input_shape = (img_rows, img_cols, 1)

#folder = os.path.abspath(os.getcwd())
folder= "/home/user/Desktop/vamsi/White_box_Attack/MNIST_FINAL"

def temperature_cross_entropy(gt, pred):
    loss = tf.nn.softmax_cross_entropy_with_logits(labels=gt, logits=pred/20)
    return loss

# 99.33%
def initial_model(train=True, vis=False):
    model_name = os.path.join(folder, 'initial_model.h5')
    if train:
        IMAGE_SIZE = 28 # Example value, replace with your actual image size

        model = Sequential()
        model.add(Input(shape=(IMAGE_SIZE, IMAGE_SIZE, 1), name='input'))  # Input layer

        model.add(Conv2D(32, (3, 3), use_bias=False, activation='relu', kernel_regularizer=l2(1e-5), name='conv1'))
        model.add(BatchNormalization(name='bn1'))
        model.add(MaxPooling2D(pool_size=(2, 2), name='maxpool1'))  # Reduced max-pooling

        model.add(Conv2D(64, (3, 3), use_bias=False, activation='relu', kernel_regularizer=l2(1e-5), name='conv2'))
        model.add(BatchNormalization(name='bn2'))
        model.add(MaxPooling2D(pool_size=(2, 2), name='maxpool2'))  # Reduced max-pooling

        model.add(Conv2D(128, (3, 3), use_bias=False, activation='relu', kernel_regularizer=l2(1e-5), name='conv3'))
        model.add(BatchNormalization(name='bn3'))

        model.add(Conv2D(256, (3, 3), use_bias=False, activation='relu', kernel_regularizer=l2(1e-5), name='conv4'))
        model.add(BatchNormalization(name='bn4'))

        model.add(Flatten(name='flatten'))
        model.add(Dense(20, use_bias=False, activation='relu', kernel_regularizer=l2(1e-5), name='fc1'))
        model.add(Dense(10, use_bias=False, activation='softmax', name='output'))


        sgd = optimizers.SGD(lr=0.01, momentum=0.9, nesterov=True)
        model.compile(loss=temperature_cross_entropy, optimizer=sgd, metrics=['accuracy'])
        history = model.fit(x_train, y_train, batch_size=batch_size, epochs=epochs, verbose=1)

        with open(model_name[:-3]+'_history.h5', 'wb') as f:
            pkl.dump(history.history, f)
        model.save(model_name)

    if vis:
        with open(model_name[:-3]+'_history.h5', 'rb') as f:
            history = pkl.load(f)
        plt.plot(history['loss'], color=[0,158/255,115/255])
        plt.title('Initial Model Loss')
        plt.ylabel('Loss')
        plt.xlabel('Epoch')
        plt.tight_layout()
        # plt.legend(['train'], loc='upper right')
        plt.savefig('initial_model_loss.png')
        plt.close()
        # plt.show()

    model = load_model(model_name, custom_objects={'temperature_cross_entropy': temperature_cross_entropy})
    score = model.evaluate(x_test, y_test, verbose=0)
    print(f'Original Accuracy:\t{score[1]*100}')

# 99.49%
def distilled_model(train=True, vis=False):
    model_name = os.path.join(folder, 'distilled_model.h5')
    if train:
        initial_model = load_model(os.path.join(folder, 'initial_model.h5'),
            custom_objects={'temperature_cross_entropy': temperature_cross_entropy})
        y_train = initial_model.predict(x_train, batch_size=batch_size)
        y_train = softmax(y_train/20, axis=1)

        IMAGE_SIZE = 28 # Example value, replace with your actual image size

        model = Sequential()
        model.add(Input(shape=(IMAGE_SIZE, IMAGE_SIZE, 1), name='input'))  # Input layer

        model.add(Conv2D(32, (3, 3), use_bias=False, activation='relu', kernel_regularizer=l2(1e-5), name='conv1'))
        model.add(BatchNormalization(name='bn1'))
        model.add(MaxPooling2D(pool_size=(2, 2), name='maxpool1'))  # Reduced max-pooling

        model.add(Conv2D(64, (3, 3), use_bias=False, activation='relu', kernel_regularizer=l2(1e-5), name='conv2'))
        model.add(BatchNormalization(name='bn2'))
        model.add(MaxPooling2D(pool_size=(2, 2), name='maxpool2'))  # Reduced max-pooling

        model.add(Conv2D(128, (3, 3), use_bias=False, activation='relu', kernel_regularizer=l2(1e-5), name='conv3'))
        model.add(BatchNormalization(name='bn3'))

        model.add(Conv2D(256, (3, 3), use_bias=False, activation='relu', kernel_regularizer=l2(1e-5), name='conv4'))
        model.add(BatchNormalization(name='bn4'))

        model.add(Flatten(name='flatten'))
        model.add(Dense(20, use_bias=False, activation='relu', kernel_regularizer=l2(1e-5), name='fc1'))
        model.add(Dense(10, use_bias=False, activation='softmax', name='output'))


        sgd = optimizers.SGD(lr=0.01, momentum=0.9, nesterov=True)
        model.compile(loss=temperature_cross_entropy, optimizer=sgd, metrics=['accuracy'])
        history = model.fit(x_train, y_train, batch_size=batch_size, epochs=epochs, verbose=1)

        sgd = optimizers.SGD(lr=0.1,momentum=0.9, nesterov=True)
        model.compile(loss=temperature_cross_entropy, optimizer=sgd, metrics=['accuracy'])
        history = model.fit(x_train, y_train, batch_size=batch_size, epochs=epochs, verbose=1)

        with open(model_name[:-3]+'_history.h5', 'wb') as f:
            pkl.dump(history.history, f)
        model.save(model_name)

    if vis:
        with open(model_name[:-3]+'_history.h5', 'rb') as f:
            history = pkl.load(f)
        plt.plot(history['loss'], color=[0,158/255,115/255])
        plt.title('Distilled Model Loss')
        plt.ylabel('Loss')
        plt.xlabel('Epoch')
        plt.tight_layout()
        # plt.legend(['train'], loc='upper right')
        plt.savefig('distilled_model_loss.png')
        # plt.show()

    model = load_model(model_name, custom_objects={'temperature_cross_entropy': temperature_cross_entropy})
    score = model.evaluate(x_test, y_test, verbose=0)
    print(f'Distilled Accuracy:\t{score[1]*100}')


def Main():
    initial_model(train=True, vis=False)
    distilled_model(train=True, vis=False)


if __name__ == "__main__":
    Main()

2023-10-19 16:24:38.293844: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1525] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 13703 MB memory:  -> device: 0, name: NVIDIA RTX A4000, pci bus id: 0000:19:00.0, compute capability: 8.6
/home/user/anaconda3/envs/attack/lib/python3.10/site-packages/keras/optimizer_v2/gradient_descent.py:102: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(SGD, self).__init__(name, **kwargs)


Train on 48000 samples
Epoch 1/10
48000/48000 [==============================] - 6s 120us/sample - loss: 2.2862 - accuracy: 0.5562
Epoch 2/10
48000/48000 [==============================] - 5s 109us/sample - loss: 2.2733 - accuracy: 0.8172
Epoch 3/10
48000/48000 [==============================] - 5s 110us/sample - loss: 2.2682 - accuracy: 0.9107
Epoch 4/10
48000/48000 [==============================] - 5s 110us/sample - loss: 2.2652 - accuracy: 0.9541
Epoch 5/10
48000/48000 [==============================] - 5s 111us/sample - loss: 2.2643 - accuracy: 0.9629
Epoch 6/10
48000/48000 [==============================] - 5s 109us/sample - loss: 2.2637 - accuracy: 0.9679
Epoch 7/10
48000/48000 [==============================] - 5s 109us/sample - loss: 2.2633 - accuracy: 0.9729
Epoch 8/10
48000/48000 [==============================] - 5s 109us/sample - loss: 2.2631 - accuracy: 0.9761
Epoch 9/10
48000/48000 [==============================] - 5s 108us/sample - loss: 2.2629 - accuracy: 0.9779
Epoch

/home/user/anaconda3/envs/attack/lib/python3.10/site-packages/keras/engine/training_v1.py:2057: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  updates = self.state_updates


Original Accuracy:	97.75000214576721


/home/user/anaconda3/envs/attack/lib/python3.10/site-packages/keras/engine/training_v1.py:2079: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  updates=self.state_updates,


Train on 48000 samples
Epoch 1/10
48000/48000 [==============================] - 6s 131us/sample - loss: 2.3060 - accuracy: 0.1245
Epoch 2/10
48000/48000 [==============================] - 6s 119us/sample - loss: 2.3060 - accuracy: 0.1328
Epoch 3/10
48000/48000 [==============================] - 6s 118us/sample - loss: 2.3060 - accuracy: 0.1396
Epoch 4/10
48000/48000 [==============================] - 6s 117us/sample - loss: 2.3060 - accuracy: 0.1486
Epoch 5/10
48000/48000 [==============================] - 6s 118us/sample - loss: 2.3060 - accuracy: 0.1563
Epoch 6/10
48000/48000 [==============================] - 6s 117us/sample - loss: 2.3059 - accuracy: 0.1630
Epoch 7/10
48000/48000 [==============================] - 6s 118us/sample - loss: 2.3059 - accuracy: 0.1720
Epoch 8/10
48000/48000 [==============================] - 6s 119us/sample - loss: 2.3059 - accuracy: 0.1814
Epoch 9/10
48000/48000 [==============================] - 6s 120us/sample - loss: 2.3059 - accuracy: 0.1901
Epoch

In [50]:
model_name = os.path.join(folder, 'distilled_model.h5')
model = load_model(model_name, custom_objects={'temperature_cross_entropy': temperature_cross_entropy})
score = model.evaluate(x_test, y_test, verbose=0)
print(f'Distilled Accuracy:\t{score[1]*100}')


Distilled Accuracy:	84.15833115577698


In [51]:
model_name = os.path.join(folder, 'distilled_model.h5')
model = load_model(model_name, custom_objects={'temperature_cross_entropy': temperature_cross_entropy})
score = model.evaluate(x_test_adv, y_test, verbose=0)
print(f'Distilled Accuracy:\t{score[1]*100}')


Distilled Accuracy:	16.608333587646484
